In [5]:
import os
import pandas as pd
import numpy as np
from IPython.display import clear_output
from exg.realtime_exg_processor import RealtimeEXGProcessor
import json

In [6]:
#  Load the JSON data from the file
def load_finger_thresholds():
    with open('../config/finger_thresholds.json', 'r') as file:
        data = json.load(file)

    # Convert lists back to tuples if necessary
    finger_thresholds = tuple(tuple(pair) for pair in data['finger_thresholds'])

    return finger_thresholds

finger_thresholds = load_finger_thresholds()

In [ ]:
# Load the data from the data csv file
def load_data(session_folder, csv_num, exg_columns, finger_columns):
    """Loads the data from a given data csv file."""
    # Load the data with timestamp
    exg_data = pd.read_csv(session_folder + f"/exg/data_{csv_num}.csv")
    angle_data = pd.read_csv(session_folder + f"/angle/data_{csv_num}.csv")
    
    # Select the columns we need
    exg_data = exg_data[exg_columns + ["timestamp"]]
    angle_data = angle_data[finger_columns + ["timestamp"]]
    
    # Convert the timestamp to a timedelta
    exg_data['timestamp'] =  pd.to_timedelta(exg_data["timestamp"], unit='ms')
    angle_data['timestamp'] =  pd.to_timedelta(angle_data["timestamp"], unit='ms')
    
    # Calculate sample rate based on size of the data and duration
    exg_duration = (exg_data["timestamp"].iloc[-1] - exg_data["timestamp"].iloc[0]).total_seconds()
    angle_duration = (angle_data["timestamp"].iloc[-1] - angle_data["timestamp"].iloc[0]).total_seconds()
    exg_sample_rate = round(len(exg_data) / exg_duration)
    angle_sample_rate = round(len(angle_data) / angle_duration)
    
    # Process the exg data
    window_sizes = [2, 4, 8, 16, 32, 64, 128]  # Rolling window sizes in seconds
    num_channels = len(exg_columns)  # Number of channels

    # Initialize the rolling processor
    processor = RealtimeEXGProcessor(
        window_sizes=window_sizes,
        num_channels=num_channels,
        fs=exg_sample_rate
    )
    processed_exg = processor.process(exg_data[exg_columns].values)
    processed_exg = processor.results_to_df(processed_exg)
    
    # Merge the processed data with the original data
    processed_exg = pd.concat([exg_data, processed_exg], axis=1)
    
    # Calculate the tolerance for merging the data
    tolerance = round(1 / max(exg_sample_rate, angle_sample_rate) * 1000)
    
    # Merge the data
    data = pd.merge_asof(angle_data, processed_exg, on="timestamp", direction='backward', tolerance=pd.Timedelta(tolerance, unit='ms'))
    data = data.set_index("timestamp")
    
    # Remove rows with NaN values
    data = data.dropna()
    
    return data

def load_session(session_folder, exg_columns, finger_columns):
    """Loads all csv files in a given data folder."""
    # Get all csv files in the folder
    csv_files = [f for f in os.listdir(session_folder + "/exg") if f.endswith('.csv')]
    
    # Load all csv files
    session = []
    for csv in csv_files:
        csv_num = int(csv.split("_")[1].split(".")[0])
        print(f"Loaded data_{csv_num}.csv")
        data = load_data(session_folder, csv_num, exg_columns, finger_columns)
        session.append(data)
    
    return session

# Define the columns for the data
exg_channels = [1, 2, 3, 4, 5, 6, 7, 8]
finger_columns = ["thumb", "index", "middle", "pinky"]
exg_columns = []
for emg_channel in exg_channels:
    exg_columns.append(f"ch_{emg_channel}_raw")

# Load all sessions in the data/ directory
session_folder = "../data/s_0"
session = load_session(session_folder, exg_columns, finger_columns)

Loaded data_1734924201.csv
Loaded data_1734929090.csv
Loaded data_1734928399.csv
Loaded data_1734933140.csv
Loaded data_1734933250.csv
Loaded data_1734928934.csv
Loaded data_1734926129.csv
Loaded data_1734923252.csv
Loaded data_1734926287.csv
Loaded data_1734928531.csv
Loaded data_1734927766.csv
Loaded data_1734923885.csv
Loaded data_1734925882.csv
Loaded data_1734924380.csv
Loaded data_1734933013.csv
Loaded data_1734928797.csv
Loaded data_1734923762.csv
Loaded data_1734923830.csv
Loaded data_1734923549.csv
